In [1]:
import json
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [2]:
def lowpass_filter(column,dict):#column, cutoff, order, sample_rate):
    cutoff = dict['cutoff']
    order = dict['order']  
    rate = dict['rate']
    b,a = signal.butter(order,cutoff/(rate/2),btype='lowpass')
    filtered_channel = signal.filtfilt(b,a,column)
    return (filtered_channel)

def get_psd(df,fs,bin_width):
    
    #fs = len(df)/(df.index[-1]-df.index[0])    
    f, psd = signal.welch(df.to_numpy(), 
                          fs=fs, 
                          nperseg=fs/bin_width,
                          #window='hanning',
                          axis=0
                         )

    df_psd = pd.DataFrame(psd)#,columns=df.columns)
    df_psd.columns
    df_psd['Frequency (Hz)'] = f
    df_psd = df_psd.set_index('Frequency (Hz)')
    
    return df_psd[1:] #drop the first value because it makes the plots look bad and is effectively 0


In [3]:
f=open('OPPOFindX5Pro-03-12-01.json')

In [4]:
data = json.load(f)

In [5]:
print(data.keys())

dict_keys(['MotionIQ', 'rear-streaming-time', 'vehicle-class', 'suspension', 'start-time', 'fork-streamStop', 'speed-wheel-circumference', 'shock-streamStop', 'fork-sag', 'rearReboundLow', 'rear-sensor-type', 'frontModel', 'bike-name', 'shock-present', 'stop-time-str', 'frontReboundLow', 'stop-time', 'location', 'rearSpringRate', 'scalene-pivot-to-eye2', 'publisherUnlocks', 'rearDampLow', 'fork-present', 'front-streaming-time', 'frontPreload', 'fork-worst-rssi', 'sampleRate', 'frontDampLow', 'scalene-enable', 'shock-worst-rssi', 'rearDampHigh', 'rear-axle-travel', 'frontSpringRate', 'speed-wheel-magnets', 'trailConditions', 'frontSpacers', 'fork-max-travel', 'rear-curve-data-version', 'countLocationErrors', 'head-tube-angle', 'recordMode', 'rear-axle-sag', 'scalene-eye-to-eye', 'scalene-pivot-to-eye1', 'elapsed-time', 'frontSpringSetting', 'rearModel', 'fork-disconnects', 'rearTirePressure', 'fork-drops', 'rearSpringSetting', 'time-zone', 'rearSpacers', 'frontTirePressure', 'front-axle

In [6]:
gps_data = data['locations']
frame=pd.DataFrame(gps_data)
print(frame)




             tUTC   latitude     speed    altitude   longitude
0    1.678587e+09 -38.170770  2.440802  635.269043  176.302171
1    1.678587e+09 -38.170732  3.104581  635.488037  176.302190
2    1.678587e+09 -38.170693  3.526130  634.584717  176.302205
3    1.678587e+09 -38.170678  3.729211  634.529907  176.302220
4    1.678587e+09 -38.170647  3.782291  634.529907  176.302231
..            ...        ...       ...         ...         ...
299  1.678587e+09 -38.166481  0.476711  448.283936  176.308617
300  1.678587e+09 -38.166482  0.043975  448.283936  176.308617
301  1.678587e+09 -38.166483  0.002437  447.854187  176.308617
302  1.678587e+09 -38.166483  0.000726  447.693970  176.308618
303  1.678587e+09 -38.166484  0.001362  447.819519  176.308620

[304 rows x 5 columns]


In [7]:
frame['elapsed_time'] = frame['tUTC'] - frame['tUTC'][0]
print(frame)

             tUTC   latitude     speed    altitude   longitude  elapsed_time
0    1.678587e+09 -38.170770  2.440802  635.269043  176.302171         0.000
1    1.678587e+09 -38.170732  3.104581  635.488037  176.302190         1.078
2    1.678587e+09 -38.170693  3.526130  634.584717  176.302205         1.672
3    1.678587e+09 -38.170678  3.729211  634.529907  176.302220         2.270
4    1.678587e+09 -38.170647  3.782291  634.529907  176.302231         2.867
..            ...        ...       ...         ...         ...           ...
299  1.678587e+09 -38.166481  0.476711  448.283936  176.308617       233.126
300  1.678587e+09 -38.166482  0.043975  448.283936  176.308617       233.722
301  1.678587e+09 -38.166483  0.002437  447.854187  176.308617       234.273
302  1.678587e+09 -38.166483  0.000726  447.693970  176.308618       234.798
303  1.678587e+09 -38.166484  0.001362  447.819519  176.308620       235.992

[304 rows x 6 columns]


In [8]:
susp=data['suspension']
df=pd.DataFrame(susp)

In [9]:
df

,time,rearWheel,frontForce,rearForce,shock,fork
0,6159,65.39,-1.56,1.15,25.00,101.01
1,6160,65.34,-1.56,1.15,24.98,100.57
2,6161,65.34,-1.46,0.72,24.98,99.79
3,6162,65.52,-1.46,0.72,25.06,98.66
4,6163,65.84,-1.40,0.52,25.19,97.44
...,...,...,...,...,...,...
47451,53610,9.28,-0.95,0.51,3.27,28.45
47452,53611,9.33,-0.96,0.51,3.29,28.45
47453,53612,9.33,-0.96,0.51,3.29,28.45
47454,53613,9.33,-0.95,0.50,3.29,28.40


In [10]:
size=len(df.index)
timedata,interval=np.linspace(0,size/200,num=size,retstep=True, endpoint=False)
print(timedata,interval)

[0.00000e+00 5.00000e-03 1.00000e-02 ... 2.37265e+02 2.37270e+02
 2.37275e+02] 0.005


In [11]:
df['time']=np.round(timedata,3)

In [12]:
df

,time,rearWheel,frontForce,rearForce,shock,fork
0,0.000,65.39,-1.56,1.15,25.00,101.01
1,0.005,65.34,-1.56,1.15,24.98,100.57
2,0.010,65.34,-1.46,0.72,24.98,99.79
3,0.015,65.52,-1.46,0.72,25.06,98.66
4,0.020,65.84,-1.40,0.52,25.19,97.44
...,...,...,...,...,...,...
47451,237.255,9.28,-0.95,0.51,3.27,28.45
47452,237.260,9.33,-0.96,0.51,3.29,28.45
47453,237.265,9.33,-0.96,0.51,3.29,28.45
47454,237.270,9.33,-0.95,0.50,3.29,28.40


In [15]:
y= df[df["fork"] < 0 ]
y

,time,rearWheel,frontForce,rearForce,shock,fork
9357,46.785,51.47,0.0,0.41,19.35,-9.99
9359,46.795,50.64,0.0,0.41,19.01,-9.99
9584,47.920,40.06,0.0,0.59,14.83,-9.99
9586,47.930,44.32,0.0,0.59,16.50,-9.99
20602,103.010,33.49,0.0,0.00,12.28,-9.99
31466,157.330,43.05,0.0,-0.27,16.00,-9.99


In [ ]:
def avg_filter(df,list):
    for x in list:
        val = (df.iloc[x-1] + df.iloc[x+1]) / 2
# def pos_vals(x):
#     idxs=[]
#     if x>0:
#         return x
#     else:
#         return 0
# df['fork'] = df['fork'].map(pos_vals)

In [ ]:
# df['shock'] = df['shock'].map(pos_vals)
# df['rearWheel'] = df['rearWheel'].map(pos_vals)

In [ ]:


# df.where(df.fork>0,other=0,inplace=True)
# df.where(df.shock>0,other=0,inplace=True)
# df.where(df.rearWheel>0,other=0,inplace=True)
df['forkVelocity']=np.gradient(df.fork,df.time)
df['shockVelocity']=np.gradient(df.shock,df.time)
df['rearVelocity']=np.gradient(df.rearWheel,df.time)

In [ ]:

df.describe()

In [ ]:
df.idxmax()

In [ ]:
dict = {
    'cutoff' : 20,
    'order' : 10,
    'rate' : 200
}
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig=go.Figure()
#fig=px.scatter(x=df.rearWheel, y=df.rearVelocity, text=df.time)
fig.add_trace(go.Scatter(x=df.time, y=df.forkVelocity))
fig.add_trace(go.Scatter(x=frame.elapsed_time, y=frame.speed, yaxis="y1"))
fig.update_layout(
#     # title=f"Frequency analysis ",
#     # xaxis_title="Frequency (Hz)",
#     # yaxis_title="Acceleration (g^2/Hz)",
    width=800, height=600
    )
# fig.update_layout(
#     yaxis1=dict(
#         title="Position",
#         anchor="free",
#         overlaying="y",
#         side="right",
#         position=0.05
#     ))
fig.update_traces(
    marker_size=2
    ) 
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=1)

fig.append_trace(go.Scatter(
    x=df.index,
    y=df.fork
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=df.time,
    y=df.rearWheel
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=df.time,
    y=df.forkVelocity
), row=3, col=1)

fig.append_trace(go.Scatter(
    x=df.time,
    y=df.rearVelocity
), row=4, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()

In [ ]:
start=int(40 * 200)
stop=int(48 * 200)
sample= df.fork[start:stop]
time=df.time[start:stop]
power=get_psd(sample,200,0.1)
print(sample)

In [ ]:
print(power)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=power.index[1:], y=power[0]),
              )
                        
            

fig.update_layout(
    title=f"Frequency analysis ",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Movement",
    width=1000, height=600) 
#fig.update_traces(marker_line_width=0.01, selector=dict(type='scatter'))                
fig.update_xaxes(type="log")
#fig.update_yaxes(type="log")

fig.show()